# DCA

In [1]:
import numpy as np
import cplex


k = 30
m = 50
R = np.random.uniform(low = 5,high=10,size=(k,m))
h = 3

E = np.sort(R,axis=0)
E = E[k-3:]
E = np.sum(E,axis=0)

def objective(E,X):
  return np.linalg.norm(E-X.dot(R))

In [2]:
def make_team_vector(indices):
  v = np.zeros(k)
  v[indices] = 1
  return v

### Giải bằng greedy tìm global

In [3]:
# brute force find global optima :
best_team = [0,1,2]
lowest_distance = 2e8
for first in range(k-2):
  for second in range(first,k-1):
    for third in range(second,k):
      team = np.zeros(k)
      team[[first,second,third]] = 1 # vector X_i [0,...1...1...1..0]
      d = objective(E,team) # tính khoảng cách 
      if( d < lowest_distance):
        best_team = [first,second,third]
        lowest_distance = d
        
print('best team is : ',best_team,' with distance = ',lowest_distance)

best team is :  [3, 21, 26]  with distance =  42.25652484280789


### Giải G(x) với solver :

In [4]:
ub = [1.0] *k # bound của  0 <= x <=1 
lb = [0.0] *k 
A = R.dot(R.T)
names = ['x'+str(i) for i in range(k)]
linexp = cplex.SparsePair(ind=names,val=[1.0] *k)
row = [i for i in range(k)]
qmat = [[row,A[i]] for i in range(k)] # Hệ số A để truyền vào cplex 
first_part = E.dot(R.T) # Phần E.dot (R.T)


b = -2 * first_part # không gồm phần tau * xl 

p = cplex.Cplex() # make problem

p.objective.set_sense(p.objective.sense.minimize)

p.variables.add(obj=b,ub=ub,lb=lb,names=names) #  0 <= x_i <=1

p.linear_constraints.add(lin_expr=[linexp], 
                       rhs=[3.0],senses="E") # x0+x1 +... + xk = 3
p.objective.set_quadratic(qmat) # set hệ số cho A

p.solve()

CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 435
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.02 sec. (0.01 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 30
  Integer space required    = 30
  Total non-zeros in factor = 465
  Total FP ops to factor    = 9455
Tried aggregator 1 time.
QP Presolve added 0 rows and 30 columns.
Reduced QP has 31 rows, 60 columns, and 525 nonzeros.
Reduced QP objective Q matrix has 30 nonzeros.
Presolve time = 0.03 sec. (0.07 ticks)
Parallel mode: using up to 4 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 465
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.03 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 4
  Rows in Factor            = 31
  Integer space required    = 31
  Total non-zeros in factor = 496
  Total FP ops to factor    = 10416
 Itn      Pri

In [5]:
team = np.array(p.solution.get_values())
team = np.round(team).nonzero()
print(team)
print(objective(E,make_team_vector(team)))

(array([ 3, 10, 26], dtype=int64),)
42.493925644146394


Solver có khi giải được global, có khi lại ra local

### Giải bằng G(x) - h(x) với solver,  $X_i \in R$

In [6]:
def make_problem(b):
  p = cplex.Cplex() # Update b 
  p.objective.set_sense(p.objective.sense.minimize)
  p.variables.add(obj=b,ub=ub,lb=lb,names=names)
  p.linear_constraints.add(lin_expr=[linexp],
                         rhs=[3.0],senses="E")
  p.objective.set_quadratic(qmat)
  return p

In [7]:
list_tau = [0.1,1,10,20,30,50,100,1000,10000]

In [ ]:
mat_result = {}
X_0 = np.zeros(k) 

for tau in list_tau:
  dis = []
  list_teams = []
  b_0 = E.dot(R.T) - tau *(2*X_0 - 1)
  p = make_problem(b_0)  
  p.solve()
  l = 0
  while True:
    result = p.solution.get_values()
    team = np.array(result)
    team = np.round(team)
    list_teams.append(team.nonzero())
    dist = objective(E,team)
    dis.append(dist)
    if l > 1 :
      if(dist > dis[l-1]):
        print(str(l),'#######InCorrect######') # sai vì dist bắt buộc phải giảm 
        break
      else:
        if l > 3:
          if(dist == dis[l-3]):
            break # break khi distance không đổi sau 3 vòng 
    new_b = -2 * first_part - tau * (2*team - 1) # update b
    p = make_problem(new_b)
    p.solve()
    l+=1
  mat_result[tau] = list_teams

In [9]:
mat_result

{0.1: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 1: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 10: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 20: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 30: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],


In [10]:
objective(E,make_team_vector([13,18,26]))

46.66714757002837

In [11]:
objective(E,make_team_vector([12,15,23]))

50.90685274829679

### Giải G(x) - H(x), điều kiện $X_i \in N$

In [12]:
def make_problem2(b):
  p = cplex.Cplex() # Update b 
  p.objective.set_sense(p.objective.sense.minimize)
  p.variables.add(obj=b,ub=ub,lb=lb,names=names,types=['I']*k) # int
  p.linear_constraints.add(lin_expr=[linexp],
                         rhs=[3.0],senses="E")
  p.objective.set_quadratic(qmat)
  return p

In [ ]:
mat_result = {}
X_0 = np.zeros(k) 

for tau in list_tau:
  dis = []
  list_teams = []
  b_0 = E.dot(R.T) - tau *(2*X_0 - 1)
  p = make_problem2(b_0)  
  p.solve()
  l = 0
  while True:
    result = p.solution.get_values()
    team = np.array(result)
    team = np.round(team)
    list_teams.append(team.nonzero())
    dist = objective(E,team)
    dis.append(dist)
    if l > 1 :
      if(dist > dis[l-1]):
        print(str(l),'#######InCorrect######') # sai vì dist bắt buộc phải giảm 
        break
      else:
        if l > 3:
          if(dist == dis[l-3]):
            break
    new_b = -2 * first_part - tau * (2*team - 1) # update b
    p = make_problem2(new_b)
    p.solve()
    l+=1
  mat_result[tau] = list_teams

In [14]:
mat_result

{0.1: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 1: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 10: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 20: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],
 30: [(array([ 0,  4, 25], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),),
  (array([ 3, 10, 26], dtype=int64),)],


## Nhận xét : 
- Solver giải G(x) và DCA G(x)-h(x) đều ra local optima, chưa phải global
- Khi $\tau$ lớn, G(X) không có ý nghĩa 